In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.4 MB/s eta 0:00:00


In [2]:
!pip install keras

In [3]:
!pip install tensorflow

In [4]:
import numpy as np
import pandas as pd

In [5]:
from keras.models import Sequential
from keras.layers import Dense
import keras
import tensorflow

In [6]:
(train_img,train_label),(test_img,test_label) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [7]:
train_img.shape

(60000, 28, 28)

In [8]:
test_img.shape

(10000, 28, 28)

In [9]:
train_label.shape

(60000,)

In [10]:
# converting the layout of the input for the requirement
train_img = train_img.reshape(len(train_img),28,28,1)

In [11]:
test_img = test_img.reshape(len(test_img),28,28,1)

In [12]:
from keras.preprocessing.image import array_to_img

In [13]:
array_to_img(train_img[0])

In [14]:
train_label[0]

5

In [15]:
array_to_img(test_img[0])

In [20]:
test_label[0]

7

In [21]:
train_img = train_img/255.0
test_img = test_img/255.0

In [17]:
from keras_tuner import RandomSearch

In [22]:
def build_model(hp):

  model = Sequential([
      keras.layers.Conv2D(
          filters = hp.Int("filters_l1",min_value =32,max_value =64,step=5),
          kernel_size= hp.Int('Kernel_size_l1',min_value = 15,max_value = 20, step = 1),
          activation='relu',
          padding='same'
      ),
      keras.layers.Conv2D(
          filters= hp.Int('filters_12',min_value = 64,max_value = 128,step=4),
          kernel_size= hp.Int('kernel_size_l2',min_value = 3,max_value = 10,step=1),
      ),
      keras.layers.MaxPool2D(
          pool_size = 5,
          strides=1
      ),
      keras.layers.Flatten(),
      Dense(
          units= hp.Int('units_dense_1',max_value = 10,min_value =5,step = 1),
          activation= 'relu',kernel_initializer='he_normal'),
    Dense(
        units = 10, activation='softmax'
    )
  ])
  model.compile(optimizer='adam',loss=keras.losses.SparseCategoricalCrossentropy(),metrics= 'accuracy')
  return model

In [25]:
kt = RandomSearch(build_model, objective='accuracy',max_trials=5)
kt.search(train_img,train_label, epochs=5,batch_size=2000)


Trial 5 Complete [00h 00m 43s]
accuracy: 0.9833833575248718

Best accuracy So Far: 0.9833833575248718
Total elapsed time: 00h 11m 50s


In [27]:
classifier = kt.get_best_models()[0]

In [35]:
y_pred = classifier.predict(test_img)

313/313 [==============================] - 1s 3ms/step


In [34]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        8224      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 96)        150624    
                                                                 
 max_pooling2d (MaxPooling2  (None, 18, 18, 96)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 31104)             0         
                                                                 
 dense (Dense)               (None, 10)                311050    
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                        

In [36]:
y_pred

array([[1.4940757e-06, 5.9821538e-08, 7.4687620e-08, ..., 9.9985564e-01,
        3.6997017e-09, 1.4143967e-04],
       [1.7653098e-05, 6.4041810e-03, 9.9353021e-01, ..., 7.7286799e-13,
        3.1134517e-05, 6.6277601e-09],
       [9.2506030e-05, 9.9969673e-01, 4.4459357e-05, ..., 5.7719775e-05,
        2.8424192e-06, 2.8992730e-05],
       ...,
       [1.6906064e-05, 7.8231155e-09, 6.3876565e-10, ..., 1.0983247e-09,
        3.6489612e-06, 1.0989811e-08],
       [4.8976232e-09, 1.3537769e-11, 7.3507123e-10, ..., 5.5494348e-10,
        5.4186941e-05, 6.8674355e-10],
       [6.0995617e-08, 2.5657824e-08, 9.1559336e-08, ..., 9.4159858e-13,
        1.3109502e-09, 1.1671905e-07]], dtype=float32)

In [37]:
result =np.argmax(y_pred, axis=1)

In [38]:
result

array([7, 2, 1, ..., 4, 5, 6])